In [1]:
# check the Nvidia CUDA compiler driver install or not on T4 GPU of colab
!nvcc --version
# installing necessary package for running cuda kernel on the colab gpu in notebook
!pip install nvcc4jupyter --quiet

# loading the package extension"
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp12632eyi".


LU Decomposition

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>

#define N 4

__global__ void decomposeLU(float* A, float* L, float* U, int n) {
    int k = threadIdx.x;

    if (k >= n) return;

    for (int j = k; j < n; j++) {
        U[k * n + j] = A[k * n + j];
        for (int s = 0; s < k; s++)
            U[k * n + j] -= L[k * n + s] * U[s * n + j];
    }

    L[k * n + k] = 1.0f;

    for (int i = k + 1; i < n; i++) {
        L[i * n + k] = A[i * n + k];
        for (int s = 0; s < k; s++)
            L[i * n + k] -= L[i * n + s] * U[s * n + k];
        L[i * n + k] /= U[k * n + k];
    }
}

int main() {
    float A[N*N] = {
        4, 3, 2, 1,
        3, 3, 2, 1,
        2, 2, 2, 1,
        1, 1, 1, 1
    };
    float *d_A, *d_L, *d_U;
    float L[N*N] = {0};
    float U[N*N] = {0};
    int size = N * N * sizeof(float);

    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_L, size);
    cudaMalloc((void**)&d_U, size);

    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_L, L, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_U, U, size, cudaMemcpyHostToDevice);

    decomposeLU<<<1, N>>>(d_A, d_L, d_U, N);

    cudaMemcpy(L, d_L, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(U, d_U, size, cudaMemcpyDeviceToHost);

    printf("L Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", L[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    printf("\nU Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", U[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_L);
    cudaFree(d_U);

    return 0;
}


QR Decomposition - Grahm Schimdt Methdod

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>
#include <math.h>

#define N 3  // square matrix size

__global__ void gramSchmidt(float* A, float* Q, float* R, int n) {
    int k = threadIdx.x;

    if (k >= n) return;

    for (int i = 0; i < n; i++)
        Q[i * n + k] = A[i * n + k];

    for (int j = 0; j < k; j++) {
        float dot = 0.0f;
        for (int i = 0; i < n; i++)
            dot += Q[i * n + j] * A[i * n + k];
        R[j * n + k] = dot;
        for (int i = 0; i < n; i++)
            Q[i * n + k] -= dot * Q[i * n + j];
    }

    float norm = 0.0f;
    for (int i = 0; i < n; i++)
        norm += Q[i * n + k] * Q[i * n + k];
    norm = sqrtf(norm);
    R[k * n + k] = norm;

    for (int i = 0; i < n; i++)
        Q[i * n + k] /= norm;
}

int main() {
    float A[N*N] = {
    2, 3, 1,
    4, 7, 5,
    6, 18, 19
};

    float *d_A, *d_Q, *d_R;
    float Q[N*N] = {0};
    float R[N*N] = {0};
    int size = N * N * sizeof(float);

    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_Q, size);
    cudaMalloc((void**)&d_R, size);

    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Q, Q, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_R, R, size, cudaMemcpyHostToDevice);

    gramSchmidt<<<1, N>>>(d_A, d_Q, d_R, N);

    cudaMemcpy(Q, d_Q, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(R, d_R, size, cudaMemcpyDeviceToHost);

    printf("Q Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", Q[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    printf("\nR Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", R[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_Q);
    cudaFree(d_R);

    return 0;
}


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>
#include <math.h>

#define N 3

__global__ void gramSchmidt(float *A, float *Q, float *R) {
    int k = threadIdx.x;
    if (k >= N) return;

    // Copy A column k into Q column k
    for (int i = 0; i < N; i++)
        Q[i * N + k] = A[i * N + k];

    for (int j = 0; j < k; j++) {
        float dot = 0.0f;
        for (int i = 0; i < N; i++)
            dot += Q[i * N + j] * A[i * N + k];
        R[j * N + k] = dot;
        for (int i = 0; i < N; i++)
            Q[i * N + k] -= dot * Q[i * N + j];
    }

    // Compute norm of column k
    float norm = 0.0f;
    for (int i = 0; i < N; i++)
        norm += Q[i * N + k] * Q[i * N + k];
    norm = sqrtf(norm);
    R[k * N + k] = norm;

    for (int i = 0; i < N; i++)
        Q[i * N + k] /= norm;
}

int main() {
    float A[N*N] = {
        2, 4, 1,
        0, 3, 5,
        0, 0, 6
    };

    float Q[N*N] = {0};
    float R[N*N] = {0};

    float *d_A, *d_Q, *d_R;
    size_t size = N * N * sizeof(float);

    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_Q, size);
    cudaMalloc((void**)&d_R, size);

    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_Q, Q, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_R, R, size, cudaMemcpyHostToDevice);

    gramSchmidt<<<1, N>>>(d_A, d_Q, d_R);

    cudaMemcpy(Q, d_Q, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(R, d_R, size, cudaMemcpyDeviceToHost);

    printf("A Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", A[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    printf("\nQ Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", Q[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    printf("\nR Matrix:\n");
    for (int i = 0; i < N*N; i++) {
        printf("%f ", R[i]);
        if ((i+1) % N == 0) printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_Q);
    cudaFree(d_R);

    return 0;
}


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda.h>
#include <math.h>

#define N 3
#define ITER 1000
#define EPS 1e-6

__global__ void matVecMul(float* A, float* x, float* y, int n) {
    int i = threadIdx.x;
    if (i < n) {
        float sum = 0.0f;
        for (int j = 0; j < n; j++)
            sum += A[i * n + j] * x[j];
        y[i] = sum;
    }
}

__global__ void normalize(float* v, int n) {
    __shared__ float norm;
    if (threadIdx.x == 0) {
        norm = 0.0f;
        for (int i = 0; i < n; i++)
            norm += v[i] * v[i];
        norm = sqrtf(norm);
    }
    __syncthreads();
    if (threadIdx.x < n)
        v[threadIdx.x] /= norm;
}

__global__ void dotProduct(float* a, float* b, float* result, int n) {
    __shared__ float temp[N];
    int i = threadIdx.x;
    if (i < n)
        temp[i] = a[i] * b[i];
    __syncthreads();
    if (i == 0) {
        float sum = 0.0f;
        for (int j = 0; j < n; j++)
            sum += temp[j];
        *result = sum;
    }
}

int main() {
    float A[N*N] = {
        3, 1, 1,
        -1, 3, 1,
        1, 1, 3
    };
    float x[N] = {1, 0, 0};
    float *d_A, *d_x, *d_y, *d_lambda;
    int sizeA = N * N * sizeof(float);
    int sizeV = N * sizeof(float);
    float lambda = 0.0f;

    cudaMalloc((void**)&d_A, sizeA);
    cudaMalloc((void**)&d_x, sizeV);
    cudaMalloc((void**)&d_y, sizeV);
    cudaMalloc((void**)&d_lambda, sizeof(float));

    cudaMemcpy(d_A, A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_x, x, sizeV, cudaMemcpyHostToDevice);

    for (int iter = 0; iter < ITER; iter++) {
        matVecMul<<<1, N>>>(d_A, d_x, d_y, N);
        normalize<<<1, N>>>(d_y, N);
        cudaMemcpy(d_x, d_y, sizeV, cudaMemcpyDeviceToDevice);
    }

    float Av[N];
    matVecMul<<<1, N>>>(d_A, d_x, d_y, N);
    dotProduct<<<1, N>>>(d_x, d_y, d_lambda, N);
    cudaMemcpy(&lambda, d_lambda, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(x, d_x, sizeV, cudaMemcpyDeviceToHost);

    printf("Approximate Largest Singular Value: %f\n", lambda);
    printf("Corresponding Right Singular Vector (normalized):\n");
    for (int i = 0; i < N; i++)
        printf("%f\n", x[i]);

    cudaFree(d_A);
    cudaFree(d_x);
    cudaFree(d_y);
    cudaFree(d_lambda);

    return 0;
}
